In [1]:
import tensorflow as tf 
import numpy as np 
import os 
import time
import matplotlib.pyplot as plt
from keras.models import Sequential, Model,load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
# from numba import cuda 
# device = cuda.get_current_device()
# device.reset()

In [ ]:
def Conv_Identity_block(X, filter_num, first=False, Conv=True):
    X_skip = X
    
    
    ###1st layer
    if Conv and not first:
        s = 2
    else:
        s = 1
        
    X = tf.keras.layers.Conv2D(filter_num, (1,1), strides=(s, s))(X)
    X = tf.keras.layers.BatchNormalization()(X)
    X = tf.keras.layers.Activation('relu')(X)
    
    ###2nd layer
    X = tf.keras.layers.Conv2D(filter_num, (3,3), padding='same')(X)
    X = tf.keras.layers.BatchNormalization()(X)
    X = tf.keras.layers.Activation('relu')(X)
    
    ###3rd layer
    X = tf.keras.layers.Conv2D(filter_num*4, (1,1), activation='relu')(X)
    X = tf.keras.layers.BatchNormalization()(X)
    
    
    ###Conv or Identity
    if Conv:
        X_skip = tf.keras.layers.Conv2D(filter_num*4, (1,1), strides=(s,s), activation='relu')(X_skip)
        X_skip = tf.keras.layers.BatchNormalization()(X_skip)

        
    ###4th layer
    
    X = tf.keras.layers.Add()([X, X_skip])
    X = tf.keras.layers.Activation('relu')(X)
    
    
    return X

In [ ]:
###ResNet50
def ResNet50(input_shape=(224,224,3)):
    X_input = tf.keras.layers.Input(input_shape,dtype=np.float16)
    X = tf.keras.layers.ZeroPadding2D((3,3))(X_input)
    
    X = tf.keras.layers.Conv2D(64, (7,7), strides=(2,2))(X)
    X = tf.keras.layers.BatchNormalization()(X)
    X = tf.keras.layers.Activation('relu')(X)
    X = tf.keras.layers.ZeroPadding2D((1,1))(X)
    X = tf.keras.layers.MaxPooling2D((3,3), strides=2)(X)
    
    
    
    num = [3, 4, 6, 3]
    filter_size = 64
    First = True
    layer_num = 0
    
    for i in num:
        for j in range(i):
            if j == 0:
                X = Conv_Identity_block(X, filter_size, First)
                First = False
            else:
                X = Conv_Identity_block(X, filter_size, Conv=False)
        filter_size *= 2
        layer_num += 1
    
    X = tf.keras.layers.GlobalAveragePooling2D()(X)
    X = tf.keras.layers.Flatten()(X)
    X = tf.keras.layers.Dense(256, activation='relu')(X)
    X = tf.keras.layers.Dense(128, activation='relu')(X)
    output_tensor = tf.keras.layers.Dense(2, activation='softmax')(X)
    model = Model(inputs=X_input, outputs=output_tensor, name='ResNet50')
    
    return model

In [ ]:
Base_Model = ResNet50(input_shape=(224,224,3))

In [ ]:
# Base_Model.summary()

In [13]:
ds_train = tf.keras.preprocessing.image_dataset_from_directory(
    'train/',
    class_names=['cats', 'dogs'],
    color_mode='rgb',
    batch_size=32,
    image_size=(224,224),
    shuffle=True,
    seed=123,
    validation_split=0.1,
    subset='training'
)

Found 24999 files belonging to 2 classes.
Using 22500 files for training.


In [14]:
ds_validate = tf.keras.preprocessing.image_dataset_from_directory(
    'train/',
    class_names=['cats', 'dogs'],
    color_mode='rgb',
    batch_size=16,
    image_size=(224,224),
    shuffle=True,
    seed=123,
    validation_split=0.1,
    subset='validation'
)

Found 24999 files belonging to 2 classes.
Using 2499 files for validation.


In [ ]:
# train_data = ImageDataGenerator()

In [ ]:
Base_Model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(), 
                   optimizer=tf.keras.optimizers.Adam(),
                   metrics=['accuracy'])

In [ ]:
Base_Model.fit(ds_train, validation_data=ds_validate, epochs=10)

# Keras Pre-Trained Model

In [15]:
PreModel = tf.keras.applications.ResNet50(include_top=False, 
                                          weights='imagenet', input_shape=(224,224,3), 
                                          classes=1 ,classifier_activation='sigmoid')

In [16]:
for layer in PreModel.layers:
        layer.trainable=False

In [17]:
resnet_model = Sequential()
resnet_model.add(PreModel)

In [18]:
resnet_model.add(tf.keras.layers.Flatten())
resnet_model.add(tf.keras.layers.Dense(512, activation='relu'))
resnet_model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

In [ ]:
resnet_model.summary()

In [19]:
resnet_model.compile(loss=tf.keras.losses.BinaryCrossentropy(), 
                   optimizer=tf.keras.optimizers.Adam(),
                   metrics=tf.keras.metrics.Accuracy())

In [ ]:
resnet_model.fit(ds_train, validation_data=ds_validate, epochs=10)